# 정시배송여부판단

데이터 설명 : e-commerce 배송의 정시 도착여부 (1: 정시배송 0 : 정시미배송)

x_train:
[https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv](https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv)

y_train:
[https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv](https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv)

x_test:
[https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv](https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv)

x_label(평가용) :
[https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv](https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv)

데이터 출처 :
[https://www.kaggle.com/datasets/prachi13/customer-analytics](https://www.kaggle.com/datasets/prachi13/customer-analytics)
(참고, 데이터 수정)

## 하이퍼파라미터 튜닝 없이 단순 분석

In [73]:
#필요한 모듈 import
import pandas as pd
import numpy as np
import warnings

from sklearn.preprocessing import MinMaxScaler, StandardScaler
scaler_m = MinMaxScaler()
scaler_s = StandardScaler()
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
rfc = RandomForestClassifier(random_state=0,max_depth=5)

from sklearn.metrics import roc_auc_score

#데이터 불러오기
warnings.filterwarnings('ignore')
pd.set_option('max_rows',None)
pd.set_option('max_columns',None)

X_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv')
y_train = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv')
X_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv')
x_test = X_test.copy()

y_test= pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_test.csv')
df = pd.merge(X_train,y_train,on='ID')
# print(df.info())
df.Customer_care_calls.replace('$7','7',inplace=True)
X_test.Customer_care_calls.replace('$7','7',inplace=True)

# obj_df = df.select_dtypes(include='object').columns
# for col in obj_df:
#     print(pd.unique(df[col]))
    
# obj_test =X_test.select_dtypes(include='object').columns
# for col in obj_test:
#     print(pd.unique(X_test[col]))

X = df.drop(['ID','Reached.on.Time_Y.N'],axis=1)
y = df['Reached.on.Time_Y.N']
X_test = X_test.drop('ID', axis=1)
# print(X.head())

obj_X = X.select_dtypes(include='object').columns
obj_X_test = X_test.select_dtypes(include='object').columns
num_X = X.select_dtypes(include='number').columns
num_X_test = X_test.select_dtypes(include='number').columns

def Labelencoder(df):
    for col in df.select_dtypes(include='object').columns:
        df[col]= le.fit_transform(df[col])
    return df

def Scaler_m(df):
    for col in df.select_dtypes(include='number').columns:
        df[col]= scaler_m.fit_transform(df[[col]])
    return df

def Scaler_s(df):
    for col in df.select_dtypes(include='number').columns:
        df[col]= scaler_s.fit_transform(df[[col]])
    return df

X = Labelencoder(X)
X_test = Labelencoder(X_test)
X_m = Scaler_m(X)
X_test_m = Scaler_m(X_test)
X_s = Scaler_s(X)
X_test_s = Scaler_s(X_test)

X_tr, X_val, y_tr, y_val = train_test_split(X_m,y,test_size=0.2, random_state=48)
rfc.fit(X_tr, y_tr)
print(rfc.score(X_tr,y_tr))
print(rfc.score(X_val,y_val))
pred_y = rfc.predict_proba(X_val)
print(roc_auc_score(y_val,pred_y[:,1]))
pred_x_label = rfc.predict_proba(X_test_m)
# print(roc_auc_score(y_test['Reached.on.Time_Y.N'],pred_x_label[:,1]))

submission = pd.DataFrame({'ID':x_test.ID, 'pred':pred_x_label[:,1]})
submission.to_csv('004001677.csv',index=False)
check = pd.read_csv('004001677.csv')
print(check.head())


0.7019704433497537
0.6924242424242424
0.77625520110957
     ID      pred
0  6811  0.464929
1  4320  0.418599
2  5732  0.426522
3  7429  0.463241
4  2191  0.987158


## 다시

In [72]:
import pandas as pd 
import warnings 
warnings.filterwarnings('ignore')

from sklearn.preprocessing import LabelEncoder, StandardScaler, MinMaxScaler
LE = LabelEncoder()
Scaler_s = StandardScaler()
Scaler_m = MinMaxScaler()

from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier


X_traiin = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_train.csv')
y_traiin = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/y_train.csv')
X_test = pd.read_csv('https://raw.githubusercontent.com/Datamanim/datarepo/main/shipping/X_test.csv')

# print(X_train.head())
# print(X_test.head())
# print(y_train.head())
X = X_train.drop('ID',axis=1)
X_t =X_test.drop('ID',axis=1)
y = y_train['Reached.on.Time_Y.N']

# print(X.isnull().sum())
# print(X_t.isnull().sum())
# print(X.info())
# print(X_t.info())

num = X.select_dtypes(include='number').columns
obj = X.select_dtypes(include='object').columns
# print(num)
# print(X_t.select_dtypes(include='number').columns)

# print(X[num].describe())

for col in num:
    Scaler_s.fit(X[[col]])
    X[col] = Scaler_s.transform(X[[col]])
    X_t[col] = Scaler_s.transform(X_t[[col]])
# print(obj)  
# for col in obj:
#     print(X[col].unique())
    
obj_le = ['Gender']
obj_oh = [x for x in obj if x not in obj_le]
# print(obj_oh)

for col in obj_le:
    LE.fit(X[col])
    X[col]=LE.transform(X[col])
    X_t[col]=LE.transform(X_t[col])
    
X = pd.get_dummies(X,columns=obj_oh)
X_t = pd.get_dummies(X_t,columns=obj_oh)

X, X_t = X.align(X_t,join='inner',axis=1)

X_tr, X_val, y_tr, y_val = train_test_split(X,y, random_state=12, test_size=0.2)
rfc = RandomForestClassifier(random_state=1, max_depth=6,n_estimators=250)
rfc.fit(X_tr,y_tr)
predproba_y_val = rfc.predict_proba(X_val)
pred_y_val = rfc.predict(X_val)
print(rfc.score(X_tr,y_tr))
print(rfc.score(X_val,y_val))

from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_val,predproba_y_val[:,1]))

# roc_list=[]
# for a in range(0,50):
#     X_tr, X_val, y_tr, y_val = train_test_split(X,y, random_state=a, test_size=0.2)
#     rfc.fit(X_tr,y_tr)
#     predproba_y_val = rfc.predict_proba(X_val)
#     roc= roc_auc_score(y_val,predproba_y_val[:,1])
#     roc_list.append((a,roc))
# print(roc_list)

0.722243273967412
0.6901515151515152
0.7689982972558538
